In [1]:
import numpy as np
import os
from ecg.detection import oxi_peaks

ModuleNotFoundError: No module named 'ecg.Tasks'

In [3]:
path = 'C:/Users/au646069/Google Drive/ECG_root/Projects/in_progress/EmotionalMetaMemory/data/Data'

In [5]:
import os
for file in os.listdir(path):
    if file.endswith(".npy"):
        print(os.path.join("/mydir", file))

In [11]:
data = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.npy'):
            data.append(file)